# Storm Data Retrieval/Association with Moisture Vapor
This notebook focuses on the data collection/processing. It contains two main sources:

- GOES-16 satellite data (https://docs.opendata.aws/noaa-goes16/cics-readme.html)
- IBTRaCS tropical storm data (https://www.ncdc.noaa.gov/ibtracs/index.php?name=ib-v4-access)

Together, these sources form a data set that will be used to train a model to identify tropical cyclones. The main workflow is as follows:

1. Download full disk moisture vapor (mv) data (ABI channel 9) from GOES-16 via AWS
2. Parse the IBTRaCS data for storms from 2016-now. Do this by creating a dictionary where the key is a datetime and the value is information about the storm
3. Associate the storm data with the mv data to create a labeled data set.

In [1]:
# import stuff
from datetime import datetime
from tools.aws_goes import GOESArchiveDownloader, GOESProduct, save_s3_product
import xarray as xr
import metpy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import pandas as pd
from tqdm import tqdm
import os

# jupyter notebook config
# %matplotlib inline #suppress plotting

### Step 1: Get GOES-16 Data
Start by downloading hosted GOES-16 data from AWS using a the custom functions found in `aws_goes.py`. The main idea of this is to collect GOES-16 data in 12 hour increments. GOES-16 beams down full-disk data every ~10 minutes which would be a lot of data. Twice a day seems like a more reasonable resolution for our dataset but this is totally subjective.

In [2]:
# setup params
# startdate = datetime(2017, 9, 16, 23, 59, 0) hurricane maria dates
# enddate = datetime(2017, 10, 2, 23, 59, 0)
startdate = datetime(2017, 1, 1, 23, 59, 0)
enddate = datetime(2021, 1, 1, 23, 59, 0)
outpath = "/Users/rmcmahon/dev/cyclone_classifier/data/aws_download"
arc = GOESArchiveDownloader()
ABI_prods = arc.get_range(startdate, enddate, GOESProduct(typ='ABI', channel=9, sector='full'), day_length='half')#, satellite = 'goes17'))

# download data
for s3obj in tqdm(ABI_prods):
    save_s3_product(s3obj, outpath)

100%|██████████| 1460/1460 [2:29:45<00:00,  6.15s/it] 


### Step 2: Get Storm Data
Start by downloading the IBTRaCS data from the web (https://www.ncei.noaa.gov/data/international-best-track-archive-for-climate-stewardship-ibtracs/v04r00/access/netcdf/). In this case it is convenient to download the data since 1980 because our GOES-16 data starts in 2017. We then create a dictionary with `datetime` keys and storm information as the values. We simplify our data by only looking at tropical storms after ~12/2016 and storms with wind speeds > 34 knots.

In [3]:
# load storm data
stormfp = 'data/storm/IBTrACS.since1980.v04r00.nc'  # all storm data since 1980 (much smaller dataset)
stormdata = xr.open_dataset(stormfp)

# figure out bounds between storms
first_storm_times = stormdata.isel(storm=[0]).time.values
last_storm_times = stormdata.isel(storm=[len(stormdata.storm)-1]).time.values
start_time = first_storm_times[~np.isnat(first_storm_times)].min()
end_time = last_storm_times[~np.isnat(last_storm_times)].max()

# generate date range
time_range = pd.date_range(start=start_time, end=end_time, freq='3H').values
time_range = time_range.astype('datetime64[s]')  # clip off the nanoseconds

# create storm dictionary to store storms
storm_dict = dict()

In [4]:
# define some functions to help with time conversion

'''
Determine storm classification based on Saffir Simpson Scale
    Although there are several reporting agencies, we use US reporting because our GOES data is focused on full disk encompassing US 
'''
goes16_begin = 4000  # storm #4000 in ibtracs is around Dec 2016, we start here to reduce amount of storms we need to search through to generate storm_dict
for storm_id in tqdm(stormdata.storm[4000:].values):
    storm = stormdata.isel(storm=storm_id)
    storm_times = storm.time.values[~np.isnat(storm.time.values)]
    wind_vals = storm.usa_wind.values
    for time_idx, time in enumerate(storm_times):
        storm_time = pd.Timestamp(time).round('10min').to_pydatetime()
        latlonname = storm.time[time_idx].lat.values, storm.time[time_idx].lon.values, storm.name.values
        if wind_vals[time_idx] >= 34:  # >34 knots => tropical storm via Saffir-Simpson scale
            if storm_time in storm_dict:
                # key is rounded time, value is tuple (or maybe a class for readability and other features)
                # need a conditional check so we don't write in the same rounded value
                if (storm_time, latlonname) not in storm_dict.items():
                    storm_dict[storm_time].append(latlonname)
            else:
                storm_dict[storm_time] = [latlonname]

100%|██████████| 484/484 [00:34<00:00, 14.07it/s]


### Step 3: Label GOES-16 Data with Storm Data
Now that we've downloaded the data and created a dictionary of storms by time stamp, we finally create the data set. This dataset contains three types of output per GOES-16 image:

1. Unlabeled moisture vapor image
2. Labeled moisture vapor image - this has a bounding box drawn around a tropical storm (if one exists)
3. Labeled moisture vapor image with embedded data - same as the labeled mv image but also labeled with storm name/time information. 

Items 1,2 are used for training and validation of the model. Item 3 is used for model verification and training data verification. 

In [5]:
# define function to unpack storm data
def unpack_centers_names(data, patch_size):
    # use this function to unpack the tuples
    # tuples are stored like (lat, lon, name)
    lt = lambda l, t: [i[t] for i in l]
    lats = lt(data, 0)
    lons = lt(data, 1)
    names = lt(data, 2)
    
    patch_lons = [i-patch_size/2 for i in lons]
    patch_lats = [i-patch_size/2 for i in lats]
    patch_centers = list(zip(patch_lons, patch_lats))
    return patch_centers, names

# navigate and process each file that was downloaded by creating and saving matplotlib image
nc_files = [os.path.join(outpath,file) for file in os.listdir(outpath) if os.path.splitext(file)[1] == '.nc']

# define save paths and create if they don't exist
raw_path = 'data/dataset/raw'
labeled_path = 'data/dataset/labeled'
meta_path = 'data/dataset/meta'

for path in [raw_path, labeled_path, meta_path]:
    if not os.path.exists(path):
        os.makedirs(path)

# loop through files and create data set
for fp in tqdm(nc_files):
    # start by unpacking data and getting timestamp of GOES data
    ds = xr.open_dataset(fp)
    mv_time = ds.t.values
    mv_time_rounded = pd.Timestamp(mv_time).round('3H').to_pydatetime()
    timestamp_string = mv_time_rounded.strftime("%Y-%m-%d-%H-%M-%S")
    
    # next, draw goes mv image and save
    dat = ds.metpy.parse_cf('Rad')
    geos = dat.metpy.cartopy_crs
    x,y = dat.x, dat.y
    fig = plt.figure(figsize=(15, 12))
    ax = plt.axes(projection = geos)
    ds_data = ds['Rad'].data
    ax.imshow(ds_data, origin='upper', extent=(x.min(), x.max(), y.min(), y.max()), transform=geos);
    plt.savefig(os.path.join(raw_path, timestamp_string+"_raw.png"))
    
    # next draw on the storms if they exist by checking time of storm against storm_dict and save
    patch_size = 5
    storm_names = []
    if mv_time_rounded in storm_dict:
        patch_centers, storm_names = unpack_centers_names(storm_dict[mv_time_rounded], patch_size)
        for center in patch_centers:
            rect = patches.Rectangle(center, patch_size, patch_size, linewidth=1, edgecolor='r', facecolor='r', alpha=0.5, transform=ccrs.Geodetic())
            ax.add_patch(rect);
    plt.savefig(os.path.join(labeled_path, timestamp_string+"_labeled.png"));
    
    # finally add meta data
    plt.title("DATETIME:{}\nTROPICAL STORMS:{}".format(timestamp_string, [name.tolist().decode() for name in storm_names]));
    ax.coastlines(resolution='50m', color='black', linewidth=0.25);
    ax.add_feature(ccrs.cartopy.feature.STATES, linewidth=0.25);
    plt.savefig(os.path.join(meta_path, timestamp_string+"_meta.png"))

  1%|▏         | 20/1460 [01:55<2:14:16,  5.59s/it]<ipython-input-5-27b2a5c6844e>:39: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(15, 12))
 74%|███████▍  | 1083/1460 [1:42:18<37:27,  5.96s/it] /Users/rmcmahon/opt/anaconda3/envs/cyclone_classifier_env/lib/python3.9/site-packages/matplotlib/image.py:446: UserWarning: Warning: converting a masked element to nan.
  dv = np.float64(self.norm.vmax) - np.float64(self.norm.vmin)
/Users/rmcmahon/opt/anaconda3/envs/cyclone_classifier_env/lib/python3.9/site-packages/matplotlib/image.py:453: UserWarning: Warning: converting a masked element to nan.
  a_min = np.float64(newmin)
/Users/rmcmahon/opt/anaconda3/envs/cyclone_classifier_env/lib/python3.9/site-packages/matplotlib/image.py:458: UserWarning: 